In [ ]:
import pandas as pd



In [ ]:
# Read in the data
dfTrees=pd.read_csv('Public_Owned_Trees.csv')

print(dfTrees.head())

In [ ]:
dfHomes=pd.read_excel('columbushomes.xlsx')
dfHomes.head()

Split function that wil seperate site address into Number, road, type, and direction with the same attribute labels as the trees data. 

In [ ]:
def split_address(df, address_col):
    # Ensure all data in address_col are strings
    df[address_col] = df[address_col].astype(str)

    # Remove rows where address contains a hyphen
    df = df[~df[address_col].str.contains('-')]

    # Split the address into parts
    df[['ADDR_NUM', 'Rest']] = df[address_col].str.split(' ', 1, expand=True)
    df['STR_TYPE'] = df['Rest'].str[-2:]
    df['Rest'] = df['Rest'].str[:-3]

    # Replace NaN values in 'Rest' with an empty string
    df['Rest'] = df['Rest'].fillna('')

    # If the rest of the address starts with a known direction, extract it
    df.loc[df['Rest'].str.startswith(('N ', 'S ', 'E ', 'W ')), 'STR_PRE_DIR'] = df['Rest'].str[:1]
    df.loc[df['Rest'].str.startswith(('N ', 'S ', 'E ', 'W ')), 'Rest'] = df['Rest'].str[2:]

    # The rest of the address is the street name
    df['STR_NAME'] = df['Rest']
    df = df.drop(columns='Rest')

    # Convert the house numbers to integers, coercing errors to NaN
    df['ADDR_NUM'] = pd.to_numeric(df['ADDR_NUM'], errors='coerce')

    return df

In [ ]:
Split street address into street number and street name and street type

In [ ]:
# Remove rows where 'Sale Price' is 0 or more than 600000
dfHomes= dfHomes[(dfHomes['Sale Price'] > 0) & (dfHomes['Sale Price'] < 600000)]
dfHomes = split_address(dfHomes, 'Site Address')

# Save the processed dataframe to a new CSV file
dfHomes.to_csv('cbusHomes.csv')

dfHomes.head()

remove STR_SUF_DIR	HEIGHT LIFE_STAGE1
RETIRED>1972
PLANTING_DATE<2013

In [ ]:
Change homes from xlsx to csv

In [ ]:
dfHomes.to_csv('dfHomes.csv', index=False)

Next we will remove all building types that are not dwellings

In [ ]:
# Filter the dataframe for rows where 'Building Type' is 'Dwelling'
dfHomes= dfHomes[dfHomes['Building Type'] == 'Dwelling']
# Drop rows with a value in 'Site Addr Num Hi'
dfHomes= dfHomes[dfHomes['Site Addr Num Hi'].isna()]
# Display the first few rows of the filtered dataframe
dfHomes.head()

In [ ]:
dfHomes.to_csv('cbusHomes.csv')

In [ ]:
#remove columns STR_SUF_DIR,	HEIGHT, LIFE_STAGE1, and Change format of Retired and Planted from 1970/01/01 00:00:00+00 to 1970
#filter Retired after 1972 and planting date after 2013


dfTrees=dfTrees.drop(columns=['STR_SUF_DIR', 'HEIGHT', 'LIFE_STAGE1'])
dfTrees['PLANTING_DATE']=dfTrees['PLANTING_DATE'].str[:4]
dfTrees['RETIRED']=dfTrees['RETIRED'].str[:4]
dfTrees=dfTrees[dfTrees['PLANTING_DATE']<'2013']
dfTrees=dfTrees[dfTrees['RETIRED']>'1972']
dfTrees.to_csv('cbusTrees.csv')
dfTrees.head()


Checking for trees within a 2 block area of the house on the same street


In [ ]:
dfHomes=pd.read_csv('cbusHomes.csv')
dfTrees=pd.read_csv('cbusTrees.csv')

identify all houses with a tree or directly adjacent to a tree. Count the total numbers of trees that each house has near it. 

In [ ]:
# Rename the columns in dfHomes to match dfTrees
dfHomes.rename(columns={'Site Addr Num Lo': 'ADDR_NUM', 
                            'Site Addr Direction': 'STR_PRE_DIR', 
                            'Site Addr Street': 'STR_NAME', 
                            'Site Addr Suffix 1': 'STR_TYPE'}, inplace=True)

# Drop NaN values from the 'ADDR_NUM' column in both dataframes
dfHomes.dropna(subset=['ADDR_NUM'], inplace=True)
dfTrees.dropna(subset=['ADDR_NUM'], inplace=True)

# Convert 'ADDR_NUM' to integers in both dataframes
dfHomes['ADDR_NUM'] = dfHomes['ADDR_NUM'].astype(int)
dfTrees['ADDR_NUM'] = dfTrees['ADDR_NUM'].astype(int)

# Create dataframes for adjacent houses
df_trees_left = dfTrees.copy()
df_trees_left['ADDR_NUM'] -= 2

df_trees_right = dfTrees.copy()
df_trees_right['ADDR_NUM'] += 2

# Concatenate tree and adjacent house dataframes
df_adjacent = pd.concat([dfTrees, df_trees_left, df_trees_right])

# Merge with home data
df_merged = pd.merge(dfHomes, df_adjacent, on=['ADDR_NUM','STR_NAME','STR_TYPE','STR_PRE_DIR'], how='inner')

# Group by address and count trees
tree_counts = df_merged.groupby(['ADDR_NUM','STR_NAME','STR_TYPE','STR_PRE_DIR']).size().reset_index(name='treeCount')

# Merge tree counts back to original home dataframe
df_final = pd.merge(dfHomes, tree_counts, on=['ADDR_NUM','STR_NAME','STR_TYPE','STR_PRE_DIR'], how='left')

# Create a multi-index based on the address fields
tree_counts.set_index(['ADDR_NUM','STR_NAME','STR_TYPE','STR_PRE_DIR'], inplace=True)

# Map the tree counts to dfHomes dataframe using the multi-index
df_final['treeCount'] = df_final.set_index(['ADDR_NUM','STR_NAME','STR_TYPE','STR_PRE_DIR']).index.map(tree_counts['treeCount'])

# Fill NA values with 0 and convert to int
df_final['treeCount'] = df_final['treeCount'].fillna(0).astype(int)

# Export to CSV
df_final.to_csv('cbusHomesTrees.csv', index=False)

# Display the first few rows of the final dataframe
df_final.head()



We will now begin looking at the impact of public trees on houses home value.  However, we will control for the following factors through ultiple regression analysis. 
Public trees: 'treeCount'
Zip code: 'Zip Code'
Neighborhood: 'Neighborhood'
Year built: 'Year Built'
Year remodeled: 'Year Remodel'
Grade: 'Grade'
Condition: 'Condition'
Bedrooms: 'Bedrooms'
Full baths: 'Full Baths'
Half baths: 'Half Baths'
Basement: 'Basement'
Attic: 'Attic'
Heat and A/C: 'Heat and A/C'
Area finished above grade: 'Area Finished Above Grade'

In [ ]:
# Import necessary libraries
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
import numpy as np

# Select columns for regression analysis
df_regression = df_final[['Mkt Value-Total', 'treeCount', 'Zip Code', 'Neighborhood', 'Year Built', 'Year Remodel', 
                          'Grade', 'Condition', 'Bedrooms', 'Full Baths', 'Half Baths', 'Basement', 'Attic', 
                          'Heat and A/C', 'Area Finished Above Grade']].copy()

# Convert categorical variables into dummy/indicator variables (one-hot encoding)
df_regression = pd.get_dummies(df_regression, drop_first=True)

# Replace NaN values with the mean
df_regression.fillna(df_regression.mean(), inplace=True)

# Define dependent variable (y) and independent variables (X)
y = df_regression['Mkt Value-Total']
X = df_regression.drop('Mkt Value-Total', axis=1)

# Split the data into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a Linear Regression object
lm = LinearRegression()

# Train the model using the training sets
lm.fit(X_train, y_train)

# Predict on the test data
y_pred = lm.predict(X_test)

# Calculate the root mean square error
rmse = np.sqrt(metrics.mean_squared_error(y_test, y_pred))

# Get the coefficients
coefficients = pd.DataFrame(lm.coef_, X.columns, columns=['Coefficient'])

# Results
rmse, coefficients.sort_values(by='Coefficient', ascending=False)


Well, the RME was pretty horrible...so let's take a look at some other factors. We will start by using the feature importance method via a random forest to compute the importance of the features. 

In [ ]:
from sklearn.ensemble import RandomForestRegressor

# Initialize the random forest regressor
rf = RandomForestRegressor(n_estimators=100, random_state=42)

# Fit the model
rf.fit(X_train, y_train)

# Get feature importances
importances = rf.feature_importances_

# Convert the importances into one-dimensional 1darray with corresponding df column names as axis labels
f_importances = pd.Series(importances, X.columns)

# Sort the array in descending order of the importances
f_importances.sort_values(ascending=False, inplace=True)

f_importances.head(10)




In [1]:
# remove Unnamed: 0	Parcel Number		Site Addr Num Hi				Site Addr Suffix 2								Prior TIF Mkt Value-Impr								Style	Exterior Wall								Addl Fixtures	Total Fixtures				Fireplace Openings	Fireplace Stacks		Area Finished Below Grade	Area Rec Room	Area Unfinished	Attached Garage Spaces	Basement Garage Spaces	Carport Spaces
#from dfFinal
import pandas as pd
dfFinal=pd.read_csv('cbusHomesTrees.csv')
dfFinal.drop(columns=['Unnamed: 0','Parcel Number','Site Addr Num Hi','Site Addr Suffix 2','Prior TIF Mkt Value-Impr','Style','Exterior Wall','Addl Fixtures','Total Fixtures','Fireplace Openings','Fireplace Stacks','Area Finished Below Grade','Area Rec Room','Area Unfinished','Attached Garage Spaces','Basement Garage Spaces','Carport Spaces'], inplace=True)
dfFinal.to_csv('dfFinal.csv')

/tmp/ipykernel_5131/2826148202.py:4: DtypeWarning: Columns (7,11) have mixed types. Specify dtype option on import or set low_memory=False.
  dfFinal=pd.read_csv('cbusHomesTrees.csv')


At this point i realized how many neighborhoods dont have trees so we will remove those neighborhoods. 

In [3]:
# Find zip codes with total tree count 0
zip_codes_with_zero_trees = dfFinal.groupby('Zip Code')['treeCount'].sum()
zip_codes_with_zero_trees = zip_codes_with_zero_trees[zip_codes_with_zero_trees == 0].index.tolist()

# Drop rows with those zip codes
dfFinal_filtered = dfFinal[~dfFinal['Zip Code'].isin(zip_codes_with_zero_trees)]
dfFinal.to_csv('dfFinal_filtered.csv')
dfFinal_filtered.head()


,ADDR_NUM,STR_PRE_DIR,STR_NAME,STR_TYPE,Latitude,Longitude,Zip Code,Historic District,Neighborhood,Mkt Value-Total,...,Bedrooms,Family Rooms,Dining Rooms,Full Baths,Half Baths,Basement,Attic,Heat and A/C,Area Finished Above Grade,treeCount
0,86,W,PACEMONT,RD,40.0270,-83.0170,43202.0,NaN,3801.0,181700,...,2.0,NaN,1.0,1.0,NaN,FULL BASEMENT,NO ATTIC,HEAT/AIR CON,1063.0,0
1,1570,NaN,FRANKLIN,AV,39.9616,-82.9578,43205.0,NaN,1201.0,520100,...,3.0,NaN,NaN,2.0,1.0,FULL BASEMENT,NO ATTIC,HEAT/AIR CON,1869.0,0
2,1110,N,CASSADY,AV,39.9936,-82.9296,43219.0,NaN,5300.0,190900,...,NaN,2.0,0.0,1.0,NaN,NaN,NaN,NaN,2278.0,0
6,1502,NaN,WALSH,AV,39.9481,-83.0388,43223.0,NaN,9403.0,54100,...,3.0,1.0,1.0,1.0,1.0,FULL BASEMENT,NO ATTIC,HEAT/AIR CON,1008.0,0
7,652,NaN,WHITTIER,ST,39.9442,-82.9794,43206.0,NaN,1600.0,117000,...,3.0,NaN,1.0,1.0,1.0,FULL BASEMENT,NO ATTIC,HEAT/AIR CON,1280.0,0


In [4]:
import ydata_profiling  as ydp
# create a profiling report of df_final
profile = ydp.ProfileReport(dfFinal)
profile.to_file("df_final.html")



Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
f_importances.to_csv('feature_importances.csv')

Next we will use the top 10 feartures to apply a random forest model 

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
# Select top 10 features
selected_features = ['Area Finished Above Grade', 'Attic_NO ATTIC', 'Neighborhood', 'Zip Code', 'Year Built', 
                     'Heat and A/C_HEAT/AIR CON', 'Year Remodel', 'Grade_C', 'Bedrooms', 'Full Baths']

X_train_selected = X_train[selected_features]
X_test_selected = X_test[selected_features]

# Create a Random Forest Regressor object
rf_selected = RandomForestRegressor(n_estimators=100, random_state=42)

# Train the model using the training sets
rf_selected.fit(X_train_selected, y_train)

# Predict on the test data
y_pred_selected = rf_selected.predict(X_test_selected)

# Calculate the root mean square error
rmse_selected = np.sqrt(metrics.mean_squared_error(y_test, y_pred_selected))

rmse_selected


NameError: name 'X_train' is not defined

Still not great.  Let's try gradient boosting.  

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

# Create a Gradient Boosting Regressor object
gbr = GradientBoostingRegressor(n_estimators=100, random_state=42)

# Train the model using the training sets
gbr.fit(X_train_selected, y_train)

# Predict on the test data
y_pred_gbr = gbr.predict(X_test_selected)

# Calculate the root mean square error
rmse_gbr = np.sqrt(metrics.mean_squared_error(y_test, y_pred_gbr))

rmse_gbr


Xg Boost will be the next model to try. 

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Assuming `X` and `y` are your features and target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
from keras.layers import Dropout
from keras.callbacks import EarlyStopping
from keras import regularizers
from keras.models import Sequential
from keras.layers import Dense



# Adjustable parameters
num_neurons_layer1 = 2
num_neurons_layer2 = 4
num_neurons_layer3 = 8
num_neurons_layer4 = 16
num_neurons_layer5 = 32
num_epochs = 50
dropout_rate = 0.9
patience = 5
l2_reg = 0.0001

# Create a Sequential model
model = Sequential()

# Add an input layer and a hidden layer with num_neurons_layer1 neurons
model.add(Dense(num_neurons_layer1, activation='relu', kernel_regularizer=regularizers.l2(l2_reg), input_shape=(X_train_scaled.shape[1],)))
model.add(Dropout(dropout_rate))

# Add a second hidden layer with num_neurons_layer2 neurons
model.add(Dense(num_neurons_layer2, activation='relu', kernel_regularizer=regularizers.l2(l2_reg)))
model.add(Dropout(dropout_rate))

# Add a third hidden layer with num_neurons_layer3 neurons
model.add(Dense(num_neurons_layer3, activation='relu', kernel_regularizer=regularizers.l2(l2_reg)))
model.add(Dropout(dropout_rate))

# Add a fourth hidden layer with num_neurons_layer4 neurons
model.add(Dense(num_neurons_layer4, activation='relu', kernel_regularizer=regularizers.l2(l2_reg)))
model.add(Dropout(dropout_rate))

# Add a fifth hidden layer with num_neurons_layer5 neurons
model.add(Dense(num_neurons_layer5, activation='relu', kernel_regularizer=regularizers.l2(l2_reg)))
model.add(Dropout(dropout_rate))

# Add output layer
model.add(Dense(1))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Define early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=patience)

# Train the model
model.fit(X_train_scaled, y_train, epochs=num_epochs, validation_split=0.2, callbacks=[early_stopping], verbose=1)

# ... rest of your code


# Predict on the test data
y_pred_nn = model.predict(X_test_scaled)

# Calculate the root mean square error
rmse_nn = np.sqrt(metrics.mean_squared_error(y_test, y_pred_nn))

print(f'Root Mean Square Error: {rmse_nn}')
